In [5]:
import tensorflow as tf
import os 
import pandas as pd


In [7]:
import os
import json

os.environ["TF_CONFIG"] = json.dumps({
    "cluster": {
        "worker": ["192.168.100.17:24202", "192.168.100.31:27720"]
    },
    "task": {"type": "worker", "index": 0}
})

In [6]:
class load():
    def __init__(self, path):
        with tf.device('/GPU:0'):
            self.df = pd.read_csv(os.path.join(path, '_annotations.csv'))
            self.df = self.df.drop_duplicates(subset='filename').reset_index(drop=True)
            self.classes = self.df['class'].unique()
            self.Y_df = pd.get_dummies(self.df['class'])
            self.Y_df = self.Y_df.astype(int)
            self.Y_ = self.Y_df.values
            self.dataset = self.create_dataset(path, self.df['filename'], self.Y_)

    def load_image(self, image_path):
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=1)
        return image

    def create_dataset(self, path, image_paths, labels):
        image_paths = image_paths.apply(lambda x: os.path.join(path, x))
        image_paths = tf.convert_to_tensor(image_paths, dtype=tf.string)
        labels = tf.convert_to_tensor(labels, dtype=tf.int32)
        image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
        label_dataset = tf.data.Dataset.from_tensor_slices(labels)
        image_dataset = image_dataset.map(self.load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
        dataset = dataset.shuffle(buffer_size=1024).batch(32)
        dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        return dataset

In [8]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the MobileNetV2 model with pre-trained weights, excluding the top layers
base_model = MobileNetV2(input_shape=(640, 640, 3), include_top=False, weights='imagenet')

# Add new layers for our specific classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(107, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Load the datasets
train = load('train')
validation = load('valid')

# Train the model
model.fit(train.dataset, epochs=10)

Epoch 1/10
260/260 [==============================] - 360s 1s/step - loss: 1.4871 - accuracy: 0.6709
Epoch 2/10
260/260 [==============================] - 361s 1s/step - loss: 1.0603 - accuracy: 0.7383
Epoch 3/10
260/260 [==============================] - 357s 1s/step - loss: 0.9007 - accuracy: 0.7800
Epoch 4/10
260/260 [==============================] - 354s 1s/step - loss: 0.5161 - accuracy: 0.8080
Epoch 5/10
260/260 [==============================] - 355s 1s/step - loss: 0.5015 - accuracy: 0.8313
Epoch 6/10
260/260 [==============================] - 345s 1s/step - loss: 0.4995 - accuracy: 0.8422
Epoch 7/10
260/260 [==============================] - 345s 1s/step - loss: 0.4923 - accuracy: 0.8649
Epoch 8/10
260/260 [==============================] - 339s 1s/step - loss: 0.4519 - accuracy: 0.8678
Epoch 9/10
260/260 [==============================] - 336s 1s/step - loss: 0.4056 - accuracy: 0.868
Epoch 10/10
260/260 [==============================] - 334s 1s/step - loss: 0.4311 - accurac